# Data cleaning

In this notebook we will explore the dataset characteristics in order to clean and prepare it for analysis and modeling.

## Getting the data

In [1]:
import pandas as pd
import numpy as np
import pickle as pk

df_orign = pd.read_csv('data/Auto1-DS-TestData.csv')
df_orign.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


As we can see, the missing data is marked as '?' in this dataset.

Lets start by replacing it with proper missing data indicators. 

After that we can check the data types for each feature.


In [2]:
df = df_orign.copy()
df = df.replace(to_replace='?', value= np.NaN)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  164 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       203 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

In [3]:
# fixing columns names
df.columns = df.columns.str.replace("-", "_")
df.describe()

,symboling,wheel_base,length,width,height,curb_weight,engine_size,compression_ratio,city_mpg,highway_mpg
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,10.142537,25.219512,30.751220
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,3.972040,6.542142,6.886443
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,7.000000,13.000000,16.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,8.600000,19.000000,25.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,9.000000,24.000000,30.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,9.400000,30.000000,34.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,23.000000,49.000000,54.000000


## Missing data

With proper missing data indicators, we can have a better understanding of the quality of each record in the dataset and in each one of its features.

In [4]:
# looking for NAs
na_count = df.isnull().sum()/df.shape[0]
na_count[na_count >0]
# No NAs here

normalized_losses    0.200000
num_of_doors         0.009756
bore                 0.019512
stroke               0.019512
horsepower           0.009756
peak_rpm             0.009756
price                0.019512
dtype: float64

With 20% of missing data for normalized_losses, we will just drop this feature entirely as it won't help us creating any prediction model.

For the other features, as they have only a small portion of missing data, we will use them. However, we will drop the records containing missing data.

In [5]:
df.drop('normalized_losses', axis=1, inplace=True)
df.dropna(inplace=True)
na_count = df.isnull().sum()/df.shape[0]
na_count[na_count >0]

Series([], dtype: float64)

## Exploring and transforming the features

Now we will explore each feature individually.

The first thing to do is to understand what features should be considered as categories and what features should be considered as numeric. This is very important for building predictive models later.

In [6]:
temp_categories = df.select_dtypes('object').columns.tolist()
temp_categories

['make',
 'fuel_type',
 'aspiration',
 'num_of_doors',
 'body_style',
 'drive_wheels',
 'engine_location',
 'engine_type',
 'num_of_cylinders',
 'fuel_system',
 'bore',
 'stroke',
 'horsepower',
 'peak_rpm',
 'price']

In [7]:
for col in temp_categories:
    print(col)
    print(df[col].unique())

make
['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'mercedes-benz' 'mercury' 'mitsubishi' 'nissan' 'peugot'
 'plymouth' 'porsche' 'saab' 'subaru' 'toyota' 'volkswagen' 'volvo']
fuel_type
['gas' 'diesel']
aspiration
['std' 'turbo']
num_of_doors
['two' 'four']
body_style
['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
drive_wheels
['rwd' 'fwd' '4wd']
engine_location
['front' 'rear']
engine_type
['dohc' 'ohcv' 'ohc' 'l' 'ohcf']
num_of_cylinders
['four' 'six' 'five' 'three' 'twelve' 'eight']
fuel_system
['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' 'idi' 'spdi']
bore
['3.47' '2.68' '3.19' '3.13' '3.50' '3.31' '3.62' '2.91' '3.03' '2.97'
 '3.34' '3.60' '2.92' '3.15' '3.43' '3.63' '3.54' '3.08' '3.39' '3.76'
 '3.58' '3.46' '3.80' '3.78' '3.17' '3.35' '3.59' '2.99' '3.33' '3.70'
 '3.61' '3.94' '3.74' '2.54' '3.05' '3.27' '3.24' '3.01']
stroke
['2.68' '3.47' '3.40' '2.80' '3.19' '3.39' '3.03' '3.11' '3.23' '3.46'
 '3.90' '3.41' '3.07' '3.58' '4.17' '2.76' '3.15' 

In [8]:
# bore, stroke, horsepower, peak_rpm and price must be converted to numeric
for col in ['bore', 'stroke', 'horsepower', 'peak_rpm', 'price']:
    df[col] = df[col].astype('float')
    
# checking categories again
temp_categories = df.select_dtypes('object').columns.tolist()
for col in temp_categories:
    print(col)
    print(df[col].unique())

make
['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'mercedes-benz' 'mercury' 'mitsubishi' 'nissan' 'peugot'
 'plymouth' 'porsche' 'saab' 'subaru' 'toyota' 'volkswagen' 'volvo']
fuel_type
['gas' 'diesel']
aspiration
['std' 'turbo']
num_of_doors
['two' 'four']
body_style
['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
drive_wheels
['rwd' 'fwd' '4wd']
engine_location
['front' 'rear']
engine_type
['dohc' 'ohcv' 'ohc' 'l' 'ohcf']
num_of_cylinders
['four' 'six' 'five' 'three' 'twelve' 'eight']
fuel_system
['mpfi' '2bbl' 'mfi' '1bbl' 'spfi' 'idi' 'spdi']


In [9]:
# lets replace drive_wheels 4wd with fwd
df['drive_wheels'].replace('4wd', 'fwd', inplace=True)
df['drive_wheels'].unique()

array(['rwd', 'fwd'], dtype=object)

In [10]:
# now lets take a look on the 'numeric' features
temp_num = [col for col in df.columns if col not in temp_categories]
df[temp_num].describe()

,symboling,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
count,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000
mean,0.797927,98.923834,174.326425,65.893782,53.869948,2561.507772,128.124352,3.330622,3.248860,10.143627,103.481865,5099.740933,25.326425,30.787565,13285.025907
std,1.235582,6.152409,12.478593,2.137795,2.394770,526.700026,41.590452,0.272385,0.315421,3.977491,37.960107,468.694369,6.387828,6.816910,8089.082886
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,98.000000,3.150000,3.110000,8.500000,70.000000,4800.000000,19.000000,25.000000,7738.000000
50%,1.000000,97.000000,173.200000,65.400000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5100.000000,25.000000,30.000000,10245.000000
75%,2.000000,102.400000,184.600000,66.900000,55.700000,2952.000000,146.000000,3.590000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16515.000000
max,3.000000,120.900000,208.100000,72.000000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,262.000000,6600.000000,49.000000,54.000000,45400.000000


In [11]:
df[temp_num].sample(10)

,symboling,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
204,-1,109.1,188.8,68.9,55.5,3062,141,3.78,3.15,9.5,114.0,5400.0,19,25,22625.0
146,0,97.0,173.5,65.4,53.0,2290,108,3.62,2.64,9.0,82.0,4800.0,28,32,7463.0
126,3,89.5,168.9,65.0,51.6,2756,194,3.74,2.90,9.5,207.0,5900.0,17,25,32528.0
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
35,0,96.5,163.4,64.0,54.5,2010,92,2.91,3.41,9.2,76.0,6000.0,30,34,7295.0
48,0,113.0,199.6,69.6,52.8,4066,258,3.63,4.17,8.1,176.0,4750.0,15,19,35550.0
18,2,88.4,141.1,60.3,53.2,1488,61,2.91,3.03,9.5,48.0,5100.0,47,53,5151.0
171,2,98.4,176.2,65.6,52.0,2714,146,3.62,3.50,9.3,116.0,4800.0,24,30,11549.0
61,1,98.8,177.8,66.5,53.7,2385,122,3.39,3.39,8.6,84.0,4800.0,26,32,10595.0
111,0,107.9,186.7,68.4,56.7,3075,120,3.46,2.19,8.4,95.0,5000.0,19,24,15580.0


In [12]:
# symboling is discrete and clearly should be treated as category too
temp_categories.append('symboling')
temp_categories

['make',
 'fuel_type',
 'aspiration',
 'num_of_doors',
 'body_style',
 'drive_wheels',
 'engine_location',
 'engine_type',
 'num_of_cylinders',
 'fuel_system',
 'symboling']

Now that we have our categories properly addressed, lets write them down so we can use this information later.

In [13]:
with open('data/category_list', 'wb') as file:
    pk.dump(temp_categories, file)

We will now export the treated data.

Also, we will create a python script containing a function for running all the tasks we just did, so we can transform this and new data as well when the need arises.

In [14]:
df.to_csv('data/df_transformed.csv', index=False)